<a href="https://colab.research.google.com/github/duyqu-chan/THY-Travel-Datathon/blob/master/TravelDatathon_preselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#THY Travel Datathon Preselection Case Study

by Duygu Can, Meriç Pakkan, Neslihan Oflaz, Ahad Khaleghi Ardabili, Akın Erdem


Flying passengers can check-in through the web site, applications, kiosks and counters. In this case study content, the train data including the number of seven-month passenger check-in operations is provided. We want you to estimate the number of channels (column Operation_Count) in the csv file attached.

Mount drive to load the data:

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Dataset Expolaration

Upload the data by reading provided .csv files from Google Drive (change path if needed). There are 808696 samples in the training and 121921 instances in the test sets. Each has 23 features.

In [3]:
import pandas as pd
train_df = pd.read_csv("/content/drive/My Drive/Travel Datathon/Assessment Data/Assessment Train Data.csv")
result_df = pd.read_csv("/content/drive/My Drive/Travel Datathon/Assessment Data/Assessment Result File.csv")
print(train_df.shape)
print(result_df.shape)
train_df.head()

(808696, 23)
(121921, 23)


,Departure_YMD_LMT,Operation_YMD_LMT,Departure_Airport,Operation_Airport,Terminal_Number,Terminal_Name,Operation_Initials,Operation_Sonic_Code,Operation_Channel,Passenger_Title,Passenger_Gender,Inbound_Departure_Airport,Outbound_Arrival_Airport,SWC_FLY,Cabin_Class,SWC_FQTV_Member,Passenger_Baggage_Count,Passenger_Baggage_Weight,SWC_Staff,SWC_CIP_Passenger,SWC_VIP_Passenger,SWC_Has_Infant,Operation_Count
0,20190326,20190326,KDT,KDT,?,032363,KS,?,QC,MISTER,M,IST,?,1,Y,1,0,0,0,1,0,0,1
1,20180321,20180321,KDT,KDT,?,03F960,KS,?,QC,MISTER,M,?,KDT,1,Y,1,0,0,0,1,0,0,1
2,20190322,20190322,KDT,KDT,?,03AA2P,KS,Y011161,QC,MISTER,M,?,?,1,Y,0,0,0,0,0,0,0,1
3,20180515,20180515,KDT,KDT,?,0353D0,MK,?,TS,MISTER,M,?,?,1,Y,1,0,0,0,1,0,0,2
4,20190420,20190420,KDT,KDT,?,03F902,KS,Y013437,QC,MISTER,M,?,?,1,Y,0,0,0,1,0,0,0,1


Check what data types we have:

In [4]:
train_df.dtypes

Departure_YMD_LMT             int64
Operation_YMD_LMT             int64
Departure_Airport            object
Operation_Airport            object
Terminal_Number              object
Terminal_Name                object
Operation_Initials           object
Operation_Sonic_Code         object
Operation_Channel            object
Passenger_Title              object
Passenger_Gender             object
Inbound_Departure_Airport    object
Outbound_Arrival_Airport     object
SWC_FLY                       int64
Cabin_Class                  object
SWC_FQTV_Member               int64
Passenger_Baggage_Count       int64
Passenger_Baggage_Weight      int64
SWC_Staff                     int64
SWC_CIP_Passenger             int64
SWC_VIP_Passenger             int64
SWC_Has_Infant                int64
Operation_Count               int64
dtype: object

In the *Departure_YMD_LMT* and the *Operation_YMD_LMT* colums, date of departure and date of check is stored so it is convenient to convert them to date time objects.

In [0]:
train_df['Departure_YMD_LMT'] = pd.to_datetime(train_df['Departure_YMD_LMT'], format='%Y%m%d')
train_df['Operation_YMD_LMT'] = pd.to_datetime(train_df['Operation_YMD_LMT'], format='%Y%m%d')
result_df['Departure_YMD_LMT'] = pd.to_datetime(result_df['Departure_YMD_LMT'], format='%Y%m%d')
result_df['Operation_YMD_LMT'] = pd.to_datetime(result_df['Operation_YMD_LMT'], format='%Y%m%d')

In [6]:
train_df.dtypes

Departure_YMD_LMT            datetime64[ns]
Operation_YMD_LMT            datetime64[ns]
Departure_Airport                    object
Operation_Airport                    object
Terminal_Number                      object
Terminal_Name                        object
Operation_Initials                   object
Operation_Sonic_Code                 object
Operation_Channel                    object
Passenger_Title                      object
Passenger_Gender                     object
Inbound_Departure_Airport            object
Outbound_Arrival_Airport             object
SWC_FLY                               int64
Cabin_Class                          object
SWC_FQTV_Member                       int64
Passenger_Baggage_Count               int64
Passenger_Baggage_Weight              int64
SWC_Staff                             int64
SWC_CIP_Passenger                     int64
SWC_VIP_Passenger                     int64
SWC_Has_Infant                        int64
Operation_Count                 

In [7]:
train_df.describe()

,SWC_FLY,SWC_FQTV_Member,Passenger_Baggage_Count,Passenger_Baggage_Weight,SWC_Staff,SWC_CIP_Passenger,SWC_VIP_Passenger,SWC_Has_Infant,Operation_Count
count,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000
mean,0.939140,0.359679,0.655413,8.560313,0.029989,0.148156,0.008257,0.011681,1.534792
std,0.239073,0.479907,0.963677,13.667213,0.170557,0.355255,0.090489,0.107444,2.223746
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,15.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,54.000000,970.000000,1.000000,1.000000,1.000000,1.000000,129.000000


Convert *object* datatype to category when needed.

In [0]:
for col_name in train_df.columns:
    if train_df[col_name].dtype.name == 'object':
        train_df[col_name] = train_df[col_name].astype('category')
        result_df[col_name] = result_df[col_name].astype('category')

In [9]:
train_df.dtypes

Departure_YMD_LMT            datetime64[ns]
Operation_YMD_LMT            datetime64[ns]
Departure_Airport                  category
Operation_Airport                  category
Terminal_Number                    category
Terminal_Name                      category
Operation_Initials                 category
Operation_Sonic_Code               category
Operation_Channel                  category
Passenger_Title                    category
Passenger_Gender                   category
Inbound_Departure_Airport          category
Outbound_Arrival_Airport           category
SWC_FLY                               int64
Cabin_Class                        category
SWC_FQTV_Member                       int64
Passenger_Baggage_Count               int64
Passenger_Baggage_Weight              int64
SWC_Staff                             int64
SWC_CIP_Passenger                     int64
SWC_VIP_Passenger                     int64
SWC_Has_Infant                        int64
Operation_Count                 

See the unique levels of the categorical columns. *Departure Airport* has only one value called "KDT". So, it is not informative and should be dropped.

In [10]:
for col_name in train_df.columns:
    if train_df[col_name].dtype.name == 'category':
        print(col_name, ":", train_df[col_name].unique())

Departure_Airport : [KDT]
Categories (1, object): [KDT]
Operation_Airport : [KDT, IST, ZKG, EST, GZT, ..., HRK, ODS, SOF, KNN, MNL]
Length: 143
Categories (143, object): [KDT, IST, ZKG, EST, ..., ODS, SOF, KNN, MNL]
Terminal_Number : [?, P1, P2]
Categories (3, object): [?, P1, P2]
Terminal_Name : [032363, 03F960, 03AA2P, 0353D0, 03F902, ..., 0B19C9, 03554A, 0302CA, 035316, 03550B]
Length: 2685
Categories (2685, object): [032363, 03F960, 03AA2P, 0353D0, ..., 03554A, 0302CA, 035316, 03550B]
Operation_Initials : [KS, MK, SC, EY, Q7, ..., NB, YK, RW, OA, JF]
Length: 273
Categories (272, object): [KS, MK, SC, EY, ..., YK, RW, OA, JF]
Operation_Sonic_Code : [?, Y011161, Y013437, Y027434, Y013171, ..., M025037, Y019509, Y036439, Y015916, Y039203]
Length: 1614
Categories (1614, object): [?, Y011161, Y013437, Y027434, ..., Y019509, Y036439, Y015916,
                            Y039203]
Operation_Channel : [QC, TS, JW, TW, TY, ..., CC, BD, IR, IA, QR]
Length: 13
Categories (13, object): [QC, TS,

Generate *Operation_Channel_Group* as defined in the pdf file.

In [11]:
dict = {"JW": 'Online',
        "TW": 'Online',
        "TS": 'Mobile',
        "JM": 'Mobile',
        "TY":"Counter",
        "QC":"Counter",
        "SC":"Kiosks",
        "IR":"Other",
        "?":"Other",
        "IA":"Other",
        "BD":"Other",
        "CC":"Other",
        "QR":"Other",
        "QP":"Other",
        "QA":"Other"
        }
train_df['Operation_Channel_Group'] = train_df['Operation_Channel'].map(dict)
train_df['Operation_Channel_Group'].unique()

array(['Counter', 'Mobile', 'Online', 'Kiosks', 'Other'], dtype=object)

Do the same for the test set.

In [12]:
result_df['Operation_Channel_Group'] = result_df['Operation_Channel'].map(dict)
result_df['Operation_Channel_Group'].unique()

array(['Mobile', 'Counter', 'Online', 'Kiosks', 'Other'], dtype=object)

### Missing Value Handling

Column percentage of null values in the training and test sets are printed below. At first glance,there are only null values in the *Operation Initials* column, however some unknown values are encoded as "?" in the datasets.

In [13]:
(train_df.isnull().mean()*100).round(4)

Departure_YMD_LMT            0.0000
Operation_YMD_LMT            0.0000
Departure_Airport            0.0000
Operation_Airport            0.0000
Terminal_Number              0.0000
Terminal_Name                0.0000
Operation_Initials           0.0156
Operation_Sonic_Code         0.0000
Operation_Channel            0.0000
Passenger_Title              0.0000
Passenger_Gender             0.0000
Inbound_Departure_Airport    0.0000
Outbound_Arrival_Airport     0.0000
SWC_FLY                      0.0000
Cabin_Class                  0.0000
SWC_FQTV_Member              0.0000
Passenger_Baggage_Count      0.0000
Passenger_Baggage_Weight     0.0000
SWC_Staff                    0.0000
SWC_CIP_Passenger            0.0000
SWC_VIP_Passenger            0.0000
SWC_Has_Infant               0.0000
Operation_Count              0.0000
Operation_Channel_Group      0.0000
dtype: float64

In [14]:
(result_df.isnull().mean()*100).round(4)

Departure_YMD_LMT            0.0000
Operation_YMD_LMT            0.0000
Departure_Airport            0.0000
Operation_Airport            0.0000
Terminal_Number              0.0000
Terminal_Name                0.0000
Operation_Initials           0.0033
Operation_Sonic_Code         0.0000
Operation_Channel            0.0000
Passenger_Title              0.0000
Passenger_Gender             0.0000
Inbound_Departure_Airport    0.0000
Outbound_Arrival_Airport     0.0000
SWC_FLY                      0.0000
Cabin_Class                  0.0000
SWC_FQTV_Member              0.0000
Passenger_Baggage_Count      0.0000
Passenger_Baggage_Weight     0.0000
SWC_Staff                    0.0000
SWC_CIP_Passenger            0.0000
SWC_VIP_Passenger            0.0000
SWC_Has_Infant               0.0000
Operation_Count              0.0000
Operation_Channel_Group      0.0000
dtype: float64

Column based "?" occurance percentages in the training set:

In [15]:
import numpy as np
def unknown_perc(df):
  print("Column Name\t Percentage")
  for col_name in df.columns:
        if df[col_name].dtype.name == 'category' and (df[col_name] == "?").any():
          count = df[col_name].value_counts(dropna=False)['?']
          percentage = (count/len(df)*100).round(3)
          print(col_name,"\t", percentage)
  return
        
unknown_perc(train_df)

Column Name	 Percentage
Terminal_Number 	 90.618
Operation_Sonic_Code 	 78.927
Operation_Channel 	 2.112
Passenger_Title 	 2.331
Passenger_Gender 	 3.902
Inbound_Departure_Airport 	 91.338
Outbound_Arrival_Airport 	 64.422
Cabin_Class 	 3.409


Column based "?" occurance percentages in the test set:

In [16]:
unknown_perc(result_df)

Column Name	 Percentage
Terminal_Number 	 91.39
Operation_Sonic_Code 	 54.39
Operation_Channel 	 0.871
Passenger_Title 	 2.762
Passenger_Gender 	 0.076
Inbound_Departure_Airport 	 94.52
Outbound_Arrival_Airport 	 65.008


#### Generating Flags


Let's transform Operation_Sonic_Code to Operation_Sonic_Code_Flag and Terminal_Name to Terminal_Name_Flag in order to see if these variables are null or not. We choose to do this since there are so many different classes for these variables.

In [0]:
train_df['Operation_Sonic_Code_Flag'] = np.where(train_df['Operation_Sonic_Code']=='?', '0', '1')
train_df['Operation_Sonic_Code_Flag'] = train_df['Operation_Sonic_Code_Flag'].astype(int)
train_df['Terminal_Number_Flag'] = np.where(train_df['Terminal_Number']=='?', '0', '1')
train_df['Terminal_Number_Flag'] = train_df['Terminal_Number_Flag'].astype(int)
result_df['Operation_Sonic_Code_Flag'] = np.where(result_df['Operation_Sonic_Code']=='?', '0', '1')
result_df['Operation_Sonic_Code_Flag'] = result_df['Operation_Sonic_Code_Flag'].astype(int)
result_df['Terminal_Number_Flag'] = np.where(result_df['Terminal_Number']=='?', '0', '1')
result_df['Terminal_Number_Flag'] = result_df['Terminal_Number_Flag'].astype(int)

Convert "?" to NA when needed. Apart from *Inbound_Departure_Airport* and *Outbound_Arrival_Airport*, "?" is printed for unknown values (missing). For those specific columns "?" means that there is no inbound or outbound flight (direct flight), so they are encoded as a seperate class called *Unknown*.

In [0]:
import numpy as np
train_df['Terminal_Number'] = train_df['Terminal_Number'].replace('?', np.nan)
train_df['Operation_Channel'] = train_df['Operation_Channel'].replace('?', np.nan)
train_df['Passenger_Title'] = train_df['Passenger_Title'].replace('?', np.nan)
train_df['Passenger_Gender'] = train_df['Passenger_Gender'].replace('?', np.nan)
train_df['Inbound_Departure_Airport'] = train_df['Inbound_Departure_Airport'].replace('?', "Unknown")
train_df['Outbound_Arrival_Airport'] = train_df['Outbound_Arrival_Airport'].replace('?', "Unknown")
train_df['Cabin_Class'] = train_df['Cabin_Class'].replace('?', np.nan)
train_df["Operation_Initials"] = train_df["Operation_Initials"].replace("?",np.nan)
train_df["Operation_Sonic_Code"] = train_df["Operation_Sonic_Code"].replace("?",np.nan)

result_df['Terminal_Number'] = result_df['Terminal_Number'].replace('?', np.nan)
result_df['Operation_Channel'] = result_df['Operation_Channel'].replace('?', np.nan)
result_df['Passenger_Title'] = result_df['Passenger_Title'].replace('?', np.nan)
result_df['Passenger_Gender'] = result_df['Passenger_Gender'].replace('?', np.nan)
result_df['Inbound_Departure_Airport'] = result_df['Inbound_Departure_Airport'].replace('?', "Unknown")
result_df['Outbound_Arrival_Airport'] = result_df['Outbound_Arrival_Airport'].replace('?', "Unknown")
result_df['Cabin_Class'] = result_df['Cabin_Class'].replace('?', np.nan)
result_df["Operation_Initials"] = train_df["Operation_Initials"].replace("?",np.nan)
result_df["Operation_Sonic_Code"] = result_df["Operation_Sonic_Code"].replace("?",np.nan)

Now, the new missing value percentages for the training set becomes:

In [19]:
(train_df.isnull().mean()*100).round(4)

Departure_YMD_LMT             0.0000
Operation_YMD_LMT             0.0000
Departure_Airport             0.0000
Operation_Airport             0.0000
Terminal_Number              90.6176
Terminal_Name                 0.0000
Operation_Initials            0.0156
Operation_Sonic_Code         78.9266
Operation_Channel             2.1123
Passenger_Title               2.3307
Passenger_Gender              3.9025
Inbound_Departure_Airport     0.0000
Outbound_Arrival_Airport      0.0000
SWC_FLY                       0.0000
Cabin_Class                   3.4093
SWC_FQTV_Member               0.0000
Passenger_Baggage_Count       0.0000
Passenger_Baggage_Weight      0.0000
SWC_Staff                     0.0000
SWC_CIP_Passenger             0.0000
SWC_VIP_Passenger             0.0000
SWC_Has_Infant                0.0000
Operation_Count               0.0000
Operation_Channel_Group       0.0000
Operation_Sonic_Code_Flag     0.0000
Terminal_Number_Flag          0.0000
dtype: float64

and for the test set the result is:

In [20]:
(result_df.isnull().mean()*100).round(4)

Departure_YMD_LMT             0.0000
Operation_YMD_LMT             0.0000
Departure_Airport             0.0000
Operation_Airport             0.0000
Terminal_Number              91.3903
Terminal_Name                 0.0000
Operation_Initials            0.0180
Operation_Sonic_Code         54.3901
Operation_Channel             0.8711
Passenger_Title               2.7616
Passenger_Gender              0.0763
Inbound_Departure_Airport     0.0000
Outbound_Arrival_Airport      0.0000
SWC_FLY                       0.0000
Cabin_Class                   0.0000
SWC_FQTV_Member               0.0000
Passenger_Baggage_Count       0.0000
Passenger_Baggage_Weight      0.0000
SWC_Staff                     0.0000
SWC_CIP_Passenger             0.0000
SWC_VIP_Passenger             0.0000
SWC_Has_Infant                0.0000
Operation_Count               0.0000
Operation_Channel_Group       0.0000
Operation_Sonic_Code_Flag     0.0000
Terminal_Number_Flag          0.0000
dtype: float64

### Dropping Uninformative Features

Notice that *Operation Sonic Code*  has a missing value ratio of 79% and for *Terminal Number* column this ratio is even higher (>90%). With a ratio this high, we cannot impute missing values correctly. These columns should be dropped, along with the *Departure_Airport* column.


In [0]:
#train_df2 = train_df.copy()
#result_df2 = result_df.copy()
train_df = train_df.drop(columns = ["Terminal_Number", "Departure_Airport", "Operation_Sonic_Code"])
result_df = result_df.drop(columns = ["Terminal_Number", "Departure_Airport", "Operation_Sonic_Code"])

#### Imputing Passenger Gender

First, we used *Passenger Title* to impute missing values in the gender column. We replaced unknown genders whose titles are *MISTER*, *MISS* and *MISSES* with male and females, respectively. Later, we decided *Operation Channel Group* based imputation for the rest. So we grouped the dataframe by this column and found most frequent observations for the *Passenger Gender* column.

In [22]:
# Replace missing values whose titles are MISTER with M
train_df.loc[(train_df.Passenger_Gender.isna() ) & (train_df.Passenger_Title=='MISTER'),"Passenger_Gender"] = "M"
result_df.loc[(result_df.Passenger_Gender.isna() ) & (result_df.Passenger_Title=='MISTER'),"Passenger_Gender"] = "M"

# Replace missing values whose titles are MISS or MISSES with F
train_df.loc[(train_df.Passenger_Gender.isna() ) & ((train_df.Passenger_Title=='MISS') | (train_df.Passenger_Title=='MISSES')) ,"Passenger_Gender"] = "F"
result_df.loc[(result_df.Passenger_Gender.isna() ) & ((result_df.Passenger_Title=='MISS') | (result_df.Passenger_Title=='MISSES')) ,"Passenger_Gender"] = "F"

sum(train_df["Passenger_Gender"].isnull())

981

There are 981 missing values left in the gender column. We checked channel group based gender distribution and found that the most frequent gender is male for every channel

In [23]:
train_df.groupby("Operation_Channel_Group")['Passenger_Gender'].apply(lambda x: x.value_counts().index[0])#.reset_index()

Operation_Channel_Group
Counter    M
Kiosks     M
Mobile     M
Online     M
Other      M
Name: Passenger_Gender, dtype: object

In [24]:
train_df.groupby("Operation_Channel_Group")['Passenger_Gender'].apply(lambda x: x.value_counts())

Operation_Channel_Group       
Counter                  M        237238
                         F        140313
                         C         11334
                         F/INF      1984
                         M/INF      1187
                         C/INF         6
                         ?             0
Kiosks                   M         27454
                         F         13975
                         C          1143
                         F/INF       139
                         M/INF        99
                         C/INF         0
                         ?             0
Mobile                   M         91209
                         F         62979
                         C          4621
                         F/INF       548
                         M/INF       380
                         C/INF         0
                         ?             0
Online                   M        104584
                         F         82430
                         C

In [25]:
result_df.groupby("Operation_Channel_Group")['Passenger_Gender'].apply(lambda x: x.value_counts())

Operation_Channel_Group   
Counter                  M    32131
                         F    23568
                         C     3107
                         ?        0
Kiosks                   M     4364
                         F     2458
                         C      275
                         ?        0
Mobile                   M    13059
                         F    10883
                         C     1129
                         ?        0
Online                   M    14682
                         F    13371
                         C     1445
                         ?        0
Other                    M      679
                         F      585
                         C       92
                         ?        0
Name: Passenger_Gender, dtype: int64

Since most common gender for all channel groups is male, we imputed missing gender values with "M".

In [26]:
train_df['Passenger_Gender'] = train_df['Passenger_Gender'].replace(np.nan, "M")
train_df['Passenger_Gender'].unique()

[M, F, F/INF, M/INF, C, C/INF]
Categories (6, object): [M, F, F/INF, M/INF, C, C/INF]

In [27]:
result_df['Passenger_Gender'] = train_df['Passenger_Gender'].replace(np.nan, "M")
result_df['Passenger_Gender'].unique()

[M, F, F/INF, M/INF, C, C/INF]
Categories (6, object): [M, F, F/INF, M/INF, C, C/INF]

#### Imputing Passenger Title

Again, Operation Channel Group based imputation is employed. "MISTER" is most common tittle so we employed all the missin values with that.

In [28]:
train_df.groupby("Operation_Channel_Group")['Passenger_Title'].apply(lambda x: x.value_counts())

Operation_Channel_Group           
Counter                  MISTER       230324
                         MISS         107506
                         MISSES        42584
                         DOCTOR          157
                         PROFFESOR        10
                         ?                 0
Kiosks                   MISTER        26821
                         MISS          10749
                         MISSES         3910
                         DOCTOR            9
                         PROFFESOR         1
                         ?                 0
Mobile                   MISTER        92083
                         MISS          48132
                         MISSES        17537
                         DOCTOR           62
                         PROFFESOR         3
                         ?                 0
Online                   MISTER       105085
                         MISS          63692
                         MISSES        22288
                    

In [29]:
result_df.groupby("Operation_Channel_Group")['Passenger_Title'].apply(lambda x: x.value_counts())

Operation_Channel_Group           
Counter                  MISTER       31512
                         MISS         20004
                         MISSES        5053
                         DOCTOR          22
                         PROFFESOR        6
                         ?                0
Kiosks                   MISTER        4260
                         MISS          2087
                         MISSES         502
                         DOCTOR           2
                         PROFFESOR        0
                         ?                0
Mobile                   MISTER       13275
                         MISS          9310
                         MISSES        2116
                         DOCTOR           6
                         PROFFESOR        0
                         ?                0
Online                   MISTER       14972
                         MISS         11525
                         MISSES        2535
                         DOCTOR          

In [30]:
train_df['Passenger_Title'] = train_df['Passenger_Title'].replace(np.nan, "MISTER")
train_df['Passenger_Title'].unique()

[MISTER, MISS, MISSES, DOCTOR, PROFFESOR]
Categories (5, object): [MISTER, MISS, MISSES, DOCTOR, PROFFESOR]

In [31]:
result_df['Passenger_Title'] = result_df['Passenger_Title'].replace(np.nan, "MISTER")
result_df['Passenger_Title'].unique()

[MISS, MISTER, MISSES, DOCTOR, PROFFESOR]
Categories (5, object): [MISS, MISTER, MISSES, DOCTOR, PROFFESOR]

#### Imputing Cabin Class
Most common class is economy class for all channel groups so we imputed missing values with it.

In [32]:
train_df.groupby("Operation_Channel_Group")['Cabin_Class'].apply(lambda x: x.value_counts())

Operation_Channel_Group   
Counter                  Y    359878
                         C     24154
                         ?         0
Kiosks                   Y     40963
                         C       784
                         ?         0
Mobile                   Y    141832
                         C     10066
                         ?         0
Online                   Y    177436
                         C      8563
                         ?         0
Other                    Y     15258
                         C      2191
                         ?         0
Name: Cabin_Class, dtype: int64

In [33]:
result_df.groupby("Operation_Channel_Group")['Cabin_Class'].apply(lambda x: x.value_counts())

Operation_Channel_Group   
Counter                  Y    55315
                         C     3506
Kiosks                   Y     6964
                         C      136
Mobile                   Y    23169
                         C     1912
Online                   Y    28342
                         C     1165
Other                    Y     1029
                         C      383
Name: Cabin_Class, dtype: int64

In [34]:
train_df['Cabin_Class'] = train_df['Cabin_Class'].replace(np.nan, "Y")
train_df['Cabin_Class'].unique()

[Y, C]
Categories (2, object): [Y, C]

In [35]:
result_df['Cabin_Class'] = result_df['Cabin_Class'].replace(np.nan, "Y")
result_df['Cabin_Class'].unique()

[Y, C]
Categories (2, object): [Y, C]

#### Imputing Operation Initials

The most frequent *Operation Initials* observed in different channel groups for the training and the test set are given below.

In [36]:
train_df.groupby("Operation_Channel_Group")['Operation_Initials'].apply(lambda x: x.value_counts().index[0])

Operation_Channel_Group
Counter    KS
Kiosks     SC
Mobile     MK
Online     MK
Other      MK
Name: Operation_Initials, dtype: object

In [37]:
result_df.groupby("Operation_Channel_Group")['Operation_Initials'].apply(lambda x: x.value_counts().index[0])

Operation_Channel_Group
Counter    KS
Kiosks     KS
Mobile     KS
Online     KS
Other      KS
Name: Operation_Initials, dtype: object

The most frequent initials observed in each channel group is "KS" for test set so we replaced NA values with it.

In [38]:
result_df['Operation_Initials'] = result_df['Operation_Initials'].replace(np.nan, "KS")
result_df['Operation_Initials'].unique()

[KS, MK, SC, EY, Q7, ..., SA, O3, VO, ZF, AX]
Length: 123
Categories (123, object): [KS, MK, SC, EY, ..., O3, VO, ZF, AX]

However, we need to impute missing values differently for different channels for the training set.

In [0]:
train_df.loc[(train_df.Operation_Channel_Group == "Counter") & (train_df.Operation_Initials.isna()),"Operation_Initials"] = "KS"
train_df.loc[(train_df.Operation_Channel_Group == "Kiosks") & (train_df.Operation_Initials.isna()),"Operation_Initials"] = "SC"
train_df.loc[(train_df.Operation_Channel_Group != "Kiosks") & (train_df.Operation_Channel_Group != "Counter") & (train_df.Operation_Initials.isna()),"Operation_Initials"] = "MK"

## Feature Generation

We generate another variable to see if people checked in on-time or not. The date is given as days, so how many days there are between the flight and the check-in is added as a variable.

In [40]:
train_df['Early_Check_In'] = (train_df.Departure_YMD_LMT - train_df.Operation_YMD_LMT)
train_df['Early_Check_In'] = (train_df['Early_Check_In']/86400000000000).astype(int)
train_df['Early_Check_In'].unique()

array([   0,    1,    2, 7033,    3,   -1, 7391, 7366, 7367, 7010, 7072,
       7037, 7092, 7105, 7057, 7418, 7426, 7388, 7440, 7008, 7443, 7405,
       7002, 7016, 7415, 7384, 7100, 7455, 7386, 7067, 7378, 7101, 7398,
       7000, 7036, 7431, 7396, 7012, 7454, 7441, 7098, 7371, 7420, 7089,
       7438, 7390, 7028, 7108, 7372, 7047, 7422, 7379, 7096, 7409, 7404,
       7374, 7004, 7013, 7410, 7020, 7088, 7083, 7056, 7102, 7024, 7370,
       7095, 7110, 7051, 7049, 7109, 7421, 7414, 7063, 7452, 7376, 7424,
       7034, 7039, 7429, 6999, 7394, 7040, 7041, 7077, 7053, 7411, 7086,
       7419, 7406, 7369, 7006, 7045, 7385, 7023, 7019, 7069, 7081, 7044,
       7423, 7011, 7437, 7058, 7005, 7087, 7115, 7070, 7027, 7368, 7381,
       7425, 7074, 7389, 7025, 7054, 7076, 7380, 7106, 7432, 7449, 7387,
       7445, 7022, 7413, 7382, 7401, 7062, 7075, 7048, 7393, 7090, 7392,
       7094, 7377, 7364, 7065, 7375, 7453, 7107, 7018, 7064, 7383, 7055,
       7046, 7038, 7031, 7365, 7042, 7442, 7026, 74

There are some peculiar cases where check-in made 7000 days before in 1999!

In [41]:
train_df[train_df['Early_Check_In']>100].sort_values('Operation_YMD_LMT')

,Departure_YMD_LMT,Operation_YMD_LMT,Operation_Airport,Terminal_Name,Operation_Initials,Operation_Channel,Passenger_Title,Passenger_Gender,Inbound_Departure_Airport,Outbound_Arrival_Airport,SWC_FLY,Cabin_Class,SWC_FQTV_Member,Passenger_Baggage_Count,Passenger_Baggage_Weight,SWC_Staff,SWC_CIP_Passenger,SWC_VIP_Passenger,SWC_Has_Infant,Operation_Count,Operation_Channel_Group,Operation_Sonic_Code_Flag,Terminal_Number_Flag,Early_Check_In
469,2018-04-04,1999-01-01,TJV,03F9C5,KS,QC,MISTER,M,EST,Unknown,1,Y,1,0,0,0,0,0,0,2,Counter,0,0,7033
555391,2019-04-21,1999-01-01,JED,096D33,AJ,TY,MISSES,F,JED,Unknown,1,Y,0,1,10,0,0,0,0,1,Counter,0,0,7415
553128,2019-04-16,1999-01-01,JED,095C12,AZ,TY,MISTER,M,JED,Unknown,1,Y,1,3,35,0,1,0,0,1,Counter,0,0,7410
551608,2019-03-17,1999-01-01,ISL,03293A,KS,QC,MISTER,M,Unknown,Unknown,1,Y,0,2,28,0,0,0,0,1,Counter,1,0,7380
551287,2019-03-14,1999-01-01,ISL,035443,MK,TS,MISTER,M,Unknown,Unknown,1,Y,0,0,0,0,0,0,0,1,Mobile,0,0,7377
550406,2018-03-29,1999-01-01,TJV,0B3F66,KS,TY,MISS,F,Unknown,Unknown,1,Y,0,1,13,1,0,0,0,1,Counter,0,0,7027
549936,2019-05-31,1999-01-01,KWI,03237P,LK,QC,MISS,F,KWI,Unknown,1,Y,0,0,0,0,0,0,0,2,Counter,0,0,7455
547407,2018-06-04,1999-01-01,VKO,03543P,MK,TS,MISTER,M,EST,Unknown,1,Y,1,1,8,0,0,0,0,1,Mobile,0,0,7094
543253,2019-04-10,1999-01-01,TJV,03F901,KS,QC,MISTER,M,Unknown,NKV,1,Y,0,1,7,0,0,0,0,1,Counter,1,1,7404
541138,2018-03-12,1999-01-01,TTS,03294D,NU,QC,MISTER,M,IST,KSY,0,Y,0,0,0,0,0,0,0,1,Counter,0,1,7010


We replaced the check-in day as early, on-time and peculiar.

In [0]:
train_df.loc[train_df.Early_Check_In > 100, 'Early_Check_In_Status'] = 'Peculiar'
train_df.loc[(train_df.Early_Check_In == 0) | (train_df.Early_Check_In == -1), 'Early_Check_In_Status'] = 'On-time'
train_df.loc[(train_df.Early_Check_In == 1) | (train_df.Early_Check_In == 2) | (train_df.Early_Check_In == 3), 'Early_Check_In_Status'] = 'Early'

Do the same for the result set.

In [0]:
result_df['Early_Check_In'] = (result_df.Departure_YMD_LMT - result_df.Operation_YMD_LMT)
result_df['Early_Check_In'] = (result_df['Early_Check_In']/86400000000000).astype(int)
result_df.loc[result_df.Early_Check_In > 100, 'Early_Check_In_Status'] = 'Peculiar'
result_df.loc[(result_df.Early_Check_In == 0) | (result_df.Early_Check_In == -1), 'Early_Check_In_Status'] = 'On-time'
result_df.loc[(result_df.Early_Check_In == 1) | (result_df.Early_Check_In == 2) | (result_df.Early_Check_In == 3), 'Early_Check_In_Status'] = 'Early'

The flights with no inbound and no outbound terminal are direct flights so we generated a new colum to label those flights

In [0]:
train_df['Direct_Flight'] = np.where((train_df.Inbound_Departure_Airport == '?') & (train_df.Outbound_Arrival_Airport == '?'), '1', '0')
result_df['Direct_Flight'] = np.where((result_df.Inbound_Departure_Airport == '?') & (result_df.Outbound_Arrival_Airport == '?'), '1', '0')

We thought that may be week of day, weekend/weekday information might be usefull.

In [45]:
import datetime
#['Weekend'] = 
[x in [5,6] for x in train_df.Departure_YMD_LMT.dt.weekday]
#sales['Week'] = sales.Date.dt.week
#sales['Month'] = sales.Date.dt.month
#sales.head()

[False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,


## Encoding Categorical Features

In [46]:
train_df.dtypes

Departure_YMD_LMT            datetime64[ns]
Operation_YMD_LMT            datetime64[ns]
Operation_Airport                  category
Terminal_Name                      category
Operation_Initials                 category
Operation_Channel                  category
Passenger_Title                    category
Passenger_Gender                   category
Inbound_Departure_Airport            object
Outbound_Arrival_Airport             object
SWC_FLY                               int64
Cabin_Class                        category
SWC_FQTV_Member                       int64
Passenger_Baggage_Count               int64
Passenger_Baggage_Weight              int64
SWC_Staff                             int64
SWC_CIP_Passenger                     int64
SWC_VIP_Passenger                     int64
SWC_Has_Infant                        int64
Operation_Count                       int64
Operation_Channel_Group              object
Operation_Sonic_Code_Flag             int64
Terminal_Number_Flag            

Change object to category when needed.

In [0]:
for col_name in train_df.columns:
    if train_df[col_name].dtype.name == 'object':
        train_df[col_name] = train_df[col_name].astype('category')
        result_df[col_name] = result_df[col_name].astype('category')


Encode categorical columns

In [0]:
train_onehot = train_df.copy()
train_onehot.drop(columns = ["Operation_Initials", "Terminal_Name"], axis =1)
result_onehot = result_df.copy()
for cols in train_df.columns: #leave as traidf!!!
  if train_onehot[cols].dtype.name == 'category':
    print(cols)
    one_hot = pd.get_dummies(train_df[cols], prefix = cols)
    train_onehot = train_onehot.drop(cols,axis = 1)
    train_onehot = train_onehot.join(one_hot)
  

Operation_Airport
Terminal_Name


In [0]:
train_onehot.columns

In [0]:
train_df["Opera"]

## Feature Importance

In [0]:
train_df.columns

In [0]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1,figsize=(10, 8))
train_df["Operation_Count"].hist(bins=500, color="blue", ax=ax)


## References


1.   [Impute Missing Values](https://jamesrledoux.com/code/imputation)
2.   [Is it better to drop or impute values from data sets when applying ML, or would it be better to label them as 'missing' for categorical variables?](https://www.quora.com/Is-it-better-to-drop-or-impute-values-from-data-sets-when-applying-ML-or-would-it-be-better-to-label-them-as-missing-for-categorical-variables)

